## Tutorial to demonstrate capability of Autoscript to handle various hardware modules
- hardware
    - vacuum
    - stage
    - beam
    - detectors - HAADF, ceta
    - aberration coefficients

#### Contributor(s): Utkarsh Pratiush <utkarshp1161@gmail.com> - 23rd May 2025
#### edited - 
   

## 0. Setup imports

In [ ]:
from stemOrchestrator.logging_config   import setup_logging
data_folder  = "."
out_path = data_folder
setup_logging(out_path=out_path) 

In [ ]:
from stemOrchestrator.acquisition import TFacquisition, DMacquisition
from stemOrchestrator.simulation import DMtwin
from stemOrchestrator.process import HAADF_tiff_to_png, tiff_to_png
from autoscript_tem_microscope_client import TemMicroscopeClient
import matplotlib.pyplot as plt
import logging
plot = plt
from typing import Dict

## 01. setup the microscope configuration

In [ ]:
import os
import json
from pathlib import Path

ip = os.getenv("MICROSCOPE_IP")
port = os.getenv("MICROSCOPE_PORT")

if not ip or not port:
    secret_path = Path("../../config_secret.json")
    if secret_path.exists():
        with open(secret_path, "r") as f:
            secret = json.load(f)
            ip = ip or secret.get("ip_TF")
            port = port or secret.get("port_TF")



config = {
    "ip": ip,
    "port": port,
    "haadf_exposure": 40e-8,  # micro-seconds per pixel
    "haadf_resolution": 512, # square
    "out_path": "."
}

## 02. Initialize microscope

In [ ]:

ip = config["ip"]
port = config["port"]
haadf_exposure = config["haadf_exposure"]
out_path = config["out_path"]
haadf_resolution = config["haadf_resolution"]



microscope = TemMicroscopeClient()
microscope.connect(ip, port = port)# 7521 on velox  computer

# query state:

tf_acquisition = TFacquisition(microscope=microscope)

# put beam shift to 0,0
# tf_acquisition.move_beam_shift_positon([0, 0])


In [ ]:
tf_acquisition.ceta_cam.insertion_state

In [ ]:
tf_acquisition.open_vacuum_valves()

In [ ]:
# query vacuum state
tf_acquisition.query_vacuum_state()

In [ ]:
# query vacuum valves
tf_acquisition.query_vacuum_valves()

In [ ]:
tf_acquisition.open_vacuum_valves()

In [ ]:
# query mode of microscope
tf_acquisition.microscope.optics.optical_mode
# tf_acquisition.microscope.optics.optical_mode = "STEM"


In [ ]:
# query beam state
tf_acquisition.query_is_beam_blanked()
# tf_acquisition.unblank_beam()

In [ ]:
tf_acquisition.ceta_cam

In [ ]:
# query screen state
tf_acquisition.query_screen_postion()

In [ ]:
tf_acquisition.insert_screen()

In [ ]:
tf_acquisition.query_screen_current()

In [ ]:
tf_acquisition.query_FOV()

In [ ]:
# insert screen

In [ ]:
# measure screen current

In [ ]:
# measure screen current

In [ ]:
# query field of view
# if needed # set it # 3.279404126033114e-08, '2574.2481486146535Kx', '2.5742481486146533Mx'

In [ ]:
# where is the stage?
tf_acquisition.query_stage_position()

In [ ]:
#  query beam shift
tf_acquisition.query_beam_shift_position()

In [ ]:
# acquire haadf
haadf_np_array, haadf_tiff_name = tf_acquisition.acquire_haadf(exposure = 10e-6, resolution=512)
tf_acquisition.haadf_det.retract()    
HAADF_tiff_to_png(haadf_tiff_name)

In [ ]:
tf_acquisition.query_haadf_state()

In [ ]:
# query stage
tf_acquisition.query_stage_position()


In [ ]:
# move stage 

# tf_acquisition.move_stage_translation_absolute(x = 10.701555e-06 , y = 5.87561e-06, z =-110.776122e-06)
# tf_acquisition.move_stage_translation_relative(x =0, y=0, z = 0.0)


In [ ]:
tf_acquisition.move_stage_translation_relative(x =0.1e-6, y=0.5e-6, z = 0.0)

## 03. Aberration's

In [ ]:
import os
import json
from pathlib import Path

# Initialize to None
ip = os.getenv("MICROSCOPE_IP")
port = os.getenv("MICROSCOPE_PORT")


if not ip or not port:
    secret_path = Path("../../config_secret.json")
    if secret_path.exists():
        with open(secret_path, "r") as f:
            secret = json.load(f)
            ip = ip or secret.get("ip_CEOS_sim")
            port = port or secret.get("port_CEOS_sim")


if not ip:
    ip = input("Enter microscope IP: ")
if not port:
    port = input("Enter microscope Port: ")

try:
    port = int(port)
except ValueError:
    raise ValueError("Port must be an integer")


In [ ]:
from stemOrchestrator.acquisition import CEOSacquisition, CEOSacquisitionTCP
# ceos_acquisition = CEOSacquisition(host=ip, port=port)
ceos_acquisition = CEOSacquisitionTCP(host=ip, port=port)


In [ ]:
tableau_result = ceos_acquisition.run_tableau(tab_type="Enhanced", angle=34)# use "Fast", "Standard", "Enhanced" --angle is in mili-radian 9,18,34

In [ ]:
tableau_result 

In [ ]:
tableau_result 

In [ ]:
from autoscript_tem_microscope_client.structures import RunOptiStemSettings, RunStemAutoFocusSettings,RunAutoComaCorrectionSettings, Point, StagePosition, AdornedImage
from autoscript_tem_microscope_client.enumerations import DetectorType, CameraType, OptiStemMethod, OpticalMode

# CameraType.FLUCAM
camera_type = CameraType.BM_CETA
settings = RunAutoComaCorrectionSettings(CameraType.BM_CETA, 1024, 2, 5, tile_size=1024)

In [ ]:
tf_acquisition.optistem_c1()

## Useful Links
- [link to offline TF-Autoscript](https://www.thermofisher.com/us/en/home/electron-microscopy/products/software-em-3d-vis/software-updates.html#autoscript)
- [Autoscript examples](https://www.fei-software-center.com/tem-apps/example-scripts/)